In [25]:
import requests #-->To make HTTP requests to web pages
import pandas as pd

In [2]:
url = "https://www.scrapethissite.com/pages/frames/"
response = requests.get(url)
print(response) 

<Response [200]>


In [3]:
response.status_code  # to check the code it it is correct-->if it is 200 then is 

200

In [4]:
response.content #-->the html  (it will as bytes the saved in the memory

b'<!doctype html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Turtles All the Way Down: Frames &amp; iFrames | Scrape This Site | A public sandbox for learning web scraping</title>\n    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png" />\n\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="description" content="Some older sites might still use frames to break up thier pages. Modern ones might be using iFrames to expose data. Learn about turtles as you scrape content inside frames.">\n\n    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" rel="stylesheet" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" crossorigin="anonymous">\n    <link href=\'https://fonts.googleapis.com/css?family=Lato:400,700\' rel=\'stylesheet\' type=\'text/css\'>\n    <link rel="sty

In [5]:
response.text  #to get the exact content but as a text

'<!doctype html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Turtles All the Way Down: Frames &amp; iFrames | Scrape This Site | A public sandbox for learning web scraping</title>\n    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png" />\n\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="description" content="Some older sites might still use frames to break up thier pages. Modern ones might be using iFrames to expose data. Learn about turtles as you scrape content inside frames.">\n\n    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" rel="stylesheet" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" crossorigin="anonymous">\n    <link href=\'https://fonts.googleapis.com/css?family=Lato:400,700\' rel=\'stylesheet\' type=\'text/css\'>\n    <link rel="styl

In [6]:
response.headers #to know the header of the html page

{'Date': 'Wed, 23 Jul 2025 03:37:43 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Nel': '{"report_to":"heroku-nel","response_headers":["Via"],"max_age":3600,"success_fraction":0.01,"failure_fraction":0.1}', 'Report-To': '{"group":"heroku-nel","endpoints":[{"url":"https://nel.heroku.com/reports?s=O7Ey%2B1R6JEQ1GVLwuQwO5SNo8o844VdOY2V7eiqTNVk%3D\\u0026sid=67ff5de4-ad2b-4112-9289-cf96be89efed\\u0026ts=1753241863"}],"max_age":3600}', 'Reporting-Endpoints': 'heroku-nel="https://nel.heroku.com/reports?s=O7Ey%2B1R6JEQ1GVLwuQwO5SNo8o844VdOY2V7eiqTNVk%3D&sid=67ff5de4-ad2b-4112-9289-cf96be89efed&ts=1753241863"', 'Server': 'cloudflare', 'Via': '1.1 heroku-router', 'Cf-Cache-Status': 'DYNAMIC', 'Content-Encoding': 'gzip', 'CF-RAY': '9638240e09e170eb-MRS', 'alt-svc': 'h3=":443"; ma=86400'}

In [7]:
response.headers['Date'] #--> to get inside the key or if i want to get the exact key  #the date im openeing the page

'Wed, 23 Jul 2025 03:37:43 GMT'

In [8]:
!pip install beautifulsoup4

In [16]:
from bs4 import BeautifulSoup
import re

In [17]:
response.raise_for_status()  #raise to the error handling the error (instead of geting error it will give like try and except(show message of the error)
soup = BeautifulSoup(response.text, 'html.parser') #calling the function of beautiful soup  / i want specific content from the page so i used parser (analyzer) it is exist in the bs -->it  convert it to text

In [18]:
response.status_code

200

In [19]:
#find and add the iframe path to the main url
iframe_tag = soup.find("iframe")
if iframe_tag:
    iframe_url = "https://www.scrapethissite.com" + iframe_tag.get("src")
    print(f"Found iframe Successfully: {iframe_url}")
else:
    print("No iframe Found on the Main Page")

Found iframe Successfully: https://www.scrapethissite.com/pages/frames/?frame=i


after we have found the ifram we need to fetch it 
Why we need to fetch the iframe--> because i want to find  all the  turtle cards

In [31]:
iframe_response = requests.get(iframe_url)  # Download iframe page
iframe_response.raise_for_status()          # Stop if request failed

# Parse the iframe HTML content
iframe_soup = BeautifulSoup(iframe_response.text, "html.parser")

# Find all family cards (divs with class "family-card")
family_cards = iframe_soup.find_all("div", class_="turtle-family-card")
print(f"Found {len(family_cards)} turtle family cards.")


Found 14 turtle family cards.


In [ ]:
so there is 14 cards --> so we need to go through each turtle cards and srape the turtle details

In [32]:
base_url = "https://www.scrapethissite.com"

# we will  Create a dictionary to store all turtle data like the name the common name , when it is discovered and by who
turtle_data = {
    "turtle_name": [],
    "common_name": [],
    "discovered_year": [],
    "discovered_by": []
}

#  now we will go through each card by using loop 
for card in family_cards:
    # Extract the turtle family name
    family_name_tag = card.find("h3", class_="family-name")
    family_name = family_name_tag.text.strip() if family_name_tag else "Unknown Family"
    turtle_data["turtle_name"].append(family_name)

    # Find the "Learn more" link which contains the details of each turtles
    link_tag = card.find("a", href=True)
    if not link_tag:
        turtle_data["common_name"].append("Not Available")
        turtle_data["discovered_year"].append("Unknown")
        turtle_data["discovered_by"].append("Unknown")
        continue  # Skip to next turtle if no detail page link

    # Fetch the detail page  
    detail_url = requests.compat.urljoin(base_url, link_tag["href"])  # i have used the join url(urljoin) because it will  combines a base url with a relative url to create a full absolute url
    detail_response = requests.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.content, "html.parser")

    # Look for details in <p class="lead">
    paragraphs_turtles = detail_soup.find_all("p", class_="lead")

    common_name = "Not Available"  # defaults
    year_found = "Unknown"
    discovered_by = "Unknown"

    #  we are going to make  a loop to search for specific info in paragraphs
    for para in paragraphs:
        text = para.get_text()

        # Extract common name
        strong_tag = para.find("strong", class_="common-name")
        if strong_tag:
            common_name = strong_tag.text.strip()

        # Extract year discovered
        year_match = re.search(r"\b(1[5-9]\d{2}|20[0-2]\d|2030)\b", text)
        if year_match:
            year_found = year_match.group(0)

        # Extract discovered by name
        by_match = re.search(r"by\s+([A-Z][a-zA-Z\-]+)", text)
        if by_match:
            discovered_by = by_match.group(1)

    # Save details in the list
    turtle_data["common_name"].append(common_name)
    turtle_data["discovered_year"].append(year_found)
    turtle_data["discovered_by"].append(discovered_by)

print("The web Scraping have  completed.")


The web Scraping have  completed.


after we havescrapped the data we will print the data that i have collected

In [34]:
for i in range(len(turtle_data["turtle_name"])):
    print(f"Turtle Family: {turtle_data['turtle_name'][i]}")
    print(f"  Common Name: {turtle_data['common_name'][i]}")
    print(f"  Year Discovered: {turtle_data['discovered_year'][i]}")
    print(f"  Discovered By: {turtle_data['discovered_by'][i]}")
    print("-" * 50)


Turtle Family: Carettochelyidae
  Common Name: Madagascar big-headed, Big-headed Amazon River turtle and South American sideneck river turtles
  Year Discovered: 1869
  Discovered By: Gray
--------------------------------------------------
Turtle Family: Cheloniidae
  Common Name: Madagascar big-headed, Big-headed Amazon River turtle and South American sideneck river turtles
  Year Discovered: 1869
  Discovered By: Gray
--------------------------------------------------
Turtle Family: Chelydridae
  Common Name: Madagascar big-headed, Big-headed Amazon River turtle and South American sideneck river turtles
  Year Discovered: 1869
  Discovered By: Gray
--------------------------------------------------
Turtle Family: Dermatemydidae
  Common Name: Madagascar big-headed, Big-headed Amazon River turtle and South American sideneck river turtles
  Year Discovered: 1869
  Discovered By: Gray
--------------------------------------------------
Turtle Family: Dermochelyidae
  Common Name: Madagas

now i have print all the data of the turtles we will converted to a dataframe(table) to clean the data

In [36]:
df = pd.DataFrame(turtle_data)
df

,turtle_name,common_name,discovered_year,discovered_by
0,Carettochelyidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
1,Cheloniidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
2,Chelydridae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
3,Dermatemydidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
4,Dermochelyidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
5,Emydidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
6,Geoemydidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
7,Kinosternidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
8,Platysternidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
9,Testudinidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray


In [38]:
df.describe().T

,count,unique,top,freq
turtle_name,14,14,Carettochelyidae,1
common_name,14,1,"Madagascar big-headed, Big-headed Amazon River...",14
discovered_year,14,1,1869,14
discovered_by,14,1,Gray,14


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   turtle_name      14 non-null     object
 1   common_name      14 non-null     object
 2   discovered_year  14 non-null     object
 3   discovered_by    14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [41]:
df.isnull().sum() # to check the null values

turtle_name        0
common_name        0
discovered_year    0
discovered_by      0
dtype: int64

In [45]:
df.duplicated().sum() #to check if there is duplicated rows

np.int64(0)

In [47]:
df["discovered_year"] = df["discovered_year"].astype("Int64")

In [48]:
df

,turtle_name,common_name,discovered_year,discovered_by
0,Carettochelyidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
1,Cheloniidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
2,Chelydridae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
3,Dermatemydidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
4,Dermochelyidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
5,Emydidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
6,Geoemydidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
7,Kinosternidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
8,Platysternidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray
9,Testudinidae,"Madagascar big-headed, Big-headed Amazon River...",1869,Gray


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   turtle_name      14 non-null     object
 1   common_name      14 non-null     object
 2   discovered_year  14 non-null     Int64 
 3   discovered_by    14 non-null     object
dtypes: Int64(1), object(3)
memory usage: 594.0+ bytes
